# Prompt Chaining Multi-Agent Product Launch Campaign System
 
## Overview
This notebook demonstrates how to build a **prompt chaining sequential multi-agent system** where agents work together in a pipeline, each building upon the previous agent's output. We'll plan a comprehensive product launch campaign.

<div align="center">
<img src="lesson_2.png" alt="Alt text" width="450"/>
</div>
 
### Key Concepts Covered:
1. **Sequential Agent Chaining**: Agents process outputs in sequence
2. **Custom Plugins**: Extend kernel capabilities with external functions
3. **Temperature Tuning**: Different creativity levels for different tasks
4. **Kernel Integration**: Agents share the same kernel and plugins
5. **Progressive Refinement**: Each agent adds specialized expertise

## 1. Setup and Configuration

In [25]:
import os
from dotenv import load_dotenv

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.functions import KernelArguments
from semantic_kernel.functions import kernel_function

# Load environment variables
load_dotenv()

# Azure OpenAI Configuration
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
BASE_URL = os.getenv("URL")
API_VERSION = "2024-12-01-preview"
DEPLOYMENT = "gpt-4.1-mini"

## 2. Initialize Kernel and AI Service

In [26]:
kernel = Kernel()

# Create and register Azure OpenAI chat service
chat_service = AzureChatCompletion(
    deployment_name=DEPLOYMENT,
    api_key=AZURE_OPENAI_KEY,
    base_url=f"{BASE_URL}{DEPLOYMENT}",
    api_version=API_VERSION
)

kernel.add_service(chat_service)

## 3. Create Custom Plugin: Market Data
 
Plugins extend the kernel's capabilities. This plugin simulates market research data.
In production, this would call real APIs for market analytics, competitor data, etc.

In [27]:
class MarketDataPlugin:
    """
    Provides market intelligence and competitor analysis.
    The @kernel_function decorator makes this method available to agents.
    """
    
    @kernel_function(description="Get market trends and competitor analysis for a product category")
    async def get_market_data(self, product_category: str) -> str:
        """
        Simulates market research data retrieval.
        
        Args:
            product_category: The product category to analyze
            
        Returns:
            Formatted market intelligence report
        """
        # In production, replace with actual API calls to market research platforms
        market_data = {
            "smartphone": {
                "market_size": "$500B globally",
                "growth_rate": "5.2% YoY",
                "top_competitors": "Apple (25%), Samsung (20%), Xiaomi (12%)",
                "trends": "5G adoption, foldable screens, AI integration, sustainability focus",
                "target_demographics": "Ages 18-45, tech-savvy, urban professionals"
            },
            "smartwatch": {
                "market_size": "$96B globally",
                "growth_rate": "12.3% YoY",
                "top_competitors": "Apple Watch (36%), Samsung (10%), Fitbit (7%)",
                "trends": "Health monitoring, ECG features, longer battery life, fashion integration",
                "target_demographics": "Ages 25-55, fitness enthusiasts, health-conscious"
            },
            "default": {
                "market_size": "$100B globally",
                "growth_rate": "8% YoY",
                "top_competitors": "Market leaders hold 60% share",
                "trends": "Digital transformation, sustainability, personalization",
                "target_demographics": "Ages 25-45, middle to high income"
            }
        }
        
        data = market_data.get(product_category.lower(), market_data["default"])
        
        return (
            f"Market Intelligence Report for {product_category}:\n"
            f"• Market Size: {data['market_size']}\n"
            f"• Growth Rate: {data['growth_rate']}\n"
            f"• Competition: {data['top_competitors']}\n"
            f"• Key Trends: {data['trends']}\n"
            f"• Target Demographics: {data['target_demographics']}"
        )

# Register the plugin with the kernel
kernel.add_plugin(plugin=MarketDataPlugin(), plugin_name="MarketData")

KernelPlugin(name='MarketData', description=None, functions={'get_market_data': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='get_market_data', plugin_name='MarketData', description='Get market trends and competitor analysis for a product category', parameters=[KernelParameterMetadata(name='product_category', description=None, default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x14e10f3e0>, streaming_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x14e10f200>

## 4. Agent Factory Function
 
Creates specialized agents with customized temperature settings.
**Temperature** controls output randomness:
- **Low (0.2-0.4)**: Factual, consistent, deterministic
- **Medium (0.5-0.7)**: Balanced creativity and structure
- **High (0.8-1.0)**: Creative, diverse, exploratory

In [28]:
def create_agent(name: str, instructions: str, temperature=0.7, max_tokens=800):
    """
    Factory function to create ChatCompletionAgent instances.
    
    Args:
        name: Agent identifier
        instructions: System prompt defining the agent's role
        temperature: Creativity level (0.0-1.0)
        max_tokens: Maximum response length
    
    Returns:
        Configured ChatCompletionAgent
    """
    settings = OpenAIChatPromptExecutionSettings()
    settings.temperature = temperature
    settings.max_tokens = max_tokens
    
    return ChatCompletionAgent(
        kernel=kernel,  # Share kernel to access plugins
        service=chat_service,
        name=name,
        instructions=instructions,
        arguments=KernelArguments(settings)
    )

## 5. Define Specialized Agents
 
Each agent has a specific role in the campaign planning pipeline.
Notice how temperature increases as we move from analytical to creative tasks.

### 5.1 Strategy Agent (Analytical Foundation)

In [29]:
strategy_agent = create_agent(
    "StrategyAgent",
    "You are a product strategy consultant. "
    "Analyze the product and create a HIGH-LEVEL STRATEGY including positioning, "
    "target audience, key value propositions, and competitive advantages. "
    "Use market data to inform your strategy.",
    temperature=0.3  # Low temperature for data-driven, analytical strategy
                     # We need consistent, factual analysis grounded in market research
)

### 5.2 Marketing Agent (Campaign Structure)

In [30]:
marketing_agent = create_agent(
    "MarketingAgent",
    "You are a marketing campaign director. "
    "Build on the strategy to create a DETAILED MARKETING PLAN with specific channels, "
    "messaging frameworks, budget allocation, and timeline. "
    "Include digital, social media, PR, and traditional marketing approaches.",
    temperature=0.4  # Slightly higher to allow structured creativity in campaign design
                     # Still controlled to ensure professional marketing standards
)

### 5.3 Content Agent (Creative Execution)

In [31]:
content_agent = create_agent(
    "ContentAgent",
    "You are a creative content strategist. "
    "Design COMPELLING CONTENT IDEAS including taglines, social media concepts, "
    "video ideas, influencer partnerships, and launch event themes. "
    "Make it engaging and memorable while aligning with the marketing plan.",
    temperature=0.65  # Higher temperature for creative content generation
                      # Allows innovation while maintaining brand consistency
)

### 5.4 Integration Agent (Final Synthesis)

In [32]:
integration_agent = create_agent(
    "IntegrationAgent",
    "You are a chief marketing officer. "
    "Integrate all previous work into a COMPREHENSIVE PRODUCT LAUNCH CAMPAIGN PLAN. "
    "Ensure coherence, identify dependencies, add success metrics, "
    "and create an executive summary. Make it presentation-ready.",
    temperature=0.7  # Highest temperature for holistic integration and narrative flow
                     # Balances creativity with professional polish
)

### 6. Agent pipeline: each agent builds on the previous agent's output

In [33]:
agents = [strategy_agent, marketing_agent, content_agent, integration_agent]

## 7. Sequential Processing Pipeline
 
This is the core pattern: **agent chaining**.
Each agent:
1. Receives the cumulative output from all previous agents
2. Adds its specialized contribution
3. Passes the enhanced output to the next agent

In [34]:
async def main():
    """
    Orchestrates the multi-agent pipeline for campaign planning.
    
    Process Flow:
    1. StrategyAgent: Creates strategic foundation
    2. MarketingAgent: Adds marketing plan details
    3. ContentAgent: Develops creative content ideas
    4. IntegrationAgent: Synthesizes into final campaign plan
    """
    
    # Initial product information
    product_name = "EchoWave Pro"
    product_category = "smartwatch"
    product_description = "A premium smartwatch with advanced health monitoring, 7-day battery life, and seamless ecosystem integration"
    
    # Starting prompt for the pipeline
    current_output = (
        f"Create a comprehensive product launch campaign for:\n"
        f"Product: {product_name}\n"
        f"Category: {product_category}\n"
        f"Description: {product_description}\n"
        f"Launch Date: Q2 2025"
    )
    
    print("=" * 70)
    print("PRODUCT LAUNCH CAMPAIGN - MULTI-AGENT PLANNING SYSTEM")
    print("=" * 70)
    print(f"\nProduct: {product_name}")
    print(f"Category: {product_category}")
    print(f"Target Launch: Q2 2025")
    print("\n" + "=" * 70)
    
    # Sequential agent processing
    for agent in agents:
        print(f"\n{'='*70}")
        print(f"{agent.name} - Processing")
        print("=" * 70 + "\n")
        
        # Invoke agent with current accumulated output
        async for message in agent.invoke(current_output):
            # Each agent's output becomes input for the next agent
            current_output = str(message)
            print(current_output)
    
    # Display final integrated campaign plan
    print("\n" + "=" * 70)
    print("FINAL INTEGRATED CAMPAIGN PLAN")
    print("=" * 70 + "\n")
    print(current_output)

## 8. Run the Demo
 
**Note**: Use `await main()` in Jupyter notebooks.
Use `asyncio.run(main())` for standalone Python scripts.

In [24]:
await main()

PRODUCT LAUNCH CAMPAIGN - MULTI-AGENT PLANNING SYSTEM

Product: EchoWave Pro
Category: smartwatch
Target Launch: Q2 2025


StrategyAgent - Processing

High-Level Strategy for EchoWave Pro Launch

1. Positioning
EchoWave Pro will be positioned as the premium smartwatch for health-focused professionals and tech enthusiasts who demand advanced health monitoring, exceptional battery life, and seamless integration with their digital ecosystem. It will be marketed as the next evolution in wearable technology—blending cutting-edge health features with style and endurance.

2. Target Audience
- Primary: Ages 25-55, urban professionals, fitness enthusiasts, and health-conscious consumers seeking premium features.
- Secondary: Tech-savvy users who value ecosystem integration and extended battery performance.

3. Key Value Propositions
- Advanced Health Monitoring: State-of-the-art sensors for ECG, SpO2, sleep, and stress tracking, surpassing standard fitness trackers.
- 7-Day Battery Life: Outpe

## Key Takeaways
 
### 1. **Sequential Agent Chaining**
Agents form a **processing pipeline** where each adds specialized expertise:
- **Strategy** → **Marketing** → **Content** → **Integration**
- Each agent builds upon all previous work
- Final output is richer than any single agent could produce
 
### 2. **Temperature as a Control Mechanism**
Different tasks require different creativity levels:
 - **0.3** (Strategy): Factual, data-driven analysis
 - **0.4** (Marketing): Structured professional planning
 - **0.65** (Content): Creative ideation with brand alignment
 - **0.7** (Integration): Holistic synthesis and narrative flow
 
### 3. **Custom Plugins Extend Capabilities**
 The `MarketDataPlugin` demonstrates how to:
 - Add external data sources to the kernel
 - Make functions available to all agents
 - Simulate API calls for testing (replace with real APIs in production)
 
 ### 4. **Shared Kernel Architecture**
 All agents share the same kernel, which means:
 - They can access the same plugins
 - Consistent service configuration
 - Centralized capability management
